#Instalaciones Previas

In [ ]:
!pip3 install dwave-neal
!pip3 install qubovert
!pip3 install dwave-system
!pip install tabulate

In [ ]:
from neal import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, EmbeddingComposite
import numpy as np
import matplotlib.pyplot as plt
from time import time
import pandas as pd
from tabulate import tabulate
from dwave.cloud import Client
import time as time
import dimod
import neal
import minorminer
import dwave_networkx as dnx
from dwave.cloud import Client
from dwave.embedding import embed_qubo, unembed_sampleset
from dwave.embedding.utils import edgelist_to_adjacency
from dwave.system.samplers import DWaveSampler
from dwave.embedding.chain_breaks import majority_vote
import numpy as np
import matplotlib.pyplot as plt

## Parámetros


*   **R**: valor que toma *R* para el problema TSP. En este caso *R=3*.
*   **lis_r**: lista con los valores de *r* para facilitar operaciones.
*   **TOKEN**: token para el acceso a los computadores de D-Wave.
*   **sampler_qpu**: *sampler* o computador de D-Wave que usaremos para nuestras métricas.



In [ ]:
R=3
lis_r = range(R)
TOKEN="DEV-512e056dfd01980d70f3c29a9e1207e80f34783b"
sampler_qpu = DWaveSampler(token=TOKEN, solver={'name': 'Advantage_system6.2'})

## Grafos de las distintas topologías
Haciendo uso de la libreria *dwave_network* podemos crear grafos asociados a cada topología.

In [ ]:
#Pegasus
pegasus_graph = dnx.pegasus_graph(15)

#Chimera
chimera_graph = dnx.chimera_graph(16,16)

## Listas para guardar las métricas

In [ ]:
num_var=[]
num_qubits=[]
tiempos_ej=[]
tiempos_em=[]
chain_length=[]
num=[1,2,3,4,5,6,7,8]

[]

## Proceso de embedding o mapeo
Recordemos que el  proceso de *embedding* intenta traducir el problema lógico en un conjunto de operaciones y conexiones entre los qubits físicos, teniendo en cuenta las restricciones de conectividad y la topología del procesador.
Para ello llamaremos a la sentencia:

*embedding = minorminer.find_embedding(problem_graph, pegasus_graph)*

La variable *embedding* tendrá el resultado del mapeo en el caso de que se haya podido realizar y None en caso contrario.

Una vez tenemos el proceso realizado podemos calcular el número de qubits físicos que se están usando. Debemos tener en cuenta que realizar este proceso no nos devuelve siempre la misma solución.

In [ ]:
for n in num:
  max = 0
  lis_n = range(0,n+1)
  numVariables = len(lis_n)*len(lis_n)*len(lis_r)
  num_var.append(numVariables)
  problem_graph = nx.complete_graph(numVariables)
  t=time.time()
  embedding = minorminer.find_embedding(problem_graph, pegasus_graph)
  t=time.time()-t
  tiempos_em.append(t)


  if embedding !={}:
    num_qubits.append(sum(len(chain) for chain in embedding.values()))
    for i in embedding.values():
      if len(i) >max:
        max=len(i)
    chain_length.append(max)
  else:
    num_qubits.append(0)
    chain_length.append(0)



## Cálculo del tiempo de ejecución de un problema
Utilizando el *sampler* que hemos escogido anteriormente, llamamos a la función para estimar el timepo.



In [ ]:
for q in num_qubits:
  if q!=0:
    estimated_runtime = sampler_qpu.solver.estimate_qpu_access_time(q, num_reads=200, annealing_time=500000)
    tiempos_ej.append(estimated_runtime)
  else:
    tiempos_ej.append(0)

Pasamos el tiempo

In [ ]:
for i in range(8):
  tiempos_ej[i]=tiempos_ej[i]/1000000

## Tabla con los resultados

In [ ]:
df = pd.DataFrame({'Valores de N': num,'Número de variables':num_var,'Número de qubits':num_qubits,'Longitud de cadena máxima':chain_length,'Tiempo de embedding':tiempos_em,'Tiempo de ejecución':tiempos_ej} )
print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))

+--------------+---------------------+------------------+---------------------------+---------------------+---------------------+
| Valores de N | Número de variables | Número de qubits | Longitud de cadena máxima | Tiempo de embedding | Tiempo de ejecución |
+--------------+---------------------+------------------+---------------------------+---------------------+---------------------+
|     1.0      |        12.0         |       22.0       |            2.0            | 0.29783129692077637 | 100.01722905443252  |
|     2.0      |        27.0         |       75.0       |            3.0            | 2.6735756397247314  | 100.01774109329901  |
|     3.0      |        48.0         |      248.0       |            7.0            |  23.36939787864685  | 100.01852042843622  |
|     4.0      |        75.0         |      560.0       |           11.0            | 114.41360378265381  | 100.01896002603199  |
|     5.0      |        108.0        |      1230.0      |           16.0            | 326.

## *Solvers* disponibles

Con este fragmenteo de código podemos ver los distintos *solvers* que están disponibles para nuestro usuario.

In [ ]:
# Crear una instancia del cliente de D-Wave
client = Client(token=TOKEN)

# Obtener la lista de solvers disponibles
solver_list = client.get_solvers()
for solver in solver_list:
    print(solver)

BQMSolver(id='hybrid_binary_quadratic_model_version2')
DQMSolver(id='hybrid_discrete_quadratic_model_version1')
CQMSolver(id='hybrid_constrained_quadratic_model_version1')
StructuredSolver(id='Advantage2_prototype1.1')
StructuredSolver(id='Advantage_system6.2')
StructuredSolver(id='Advantage_system4.1')
